In [1]:
from copy import deepcopy

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [3]:
registers = {}
program = []
prog_parse = False
for line in lines:
    if len(line) == 0:
        prog_parse = True
        continue
    if not prog_parse:
        registers[line.split(":")[0][-1]] = int(line.split(":")[1])
    else:
        program = [int(i) for i in line.split(":")[1].split(",")]

In [4]:
def get_combo_operand(operand, registers):
    if operand <= 3:
        return operand 
    elif operand == 4:
        return registers["A"]
    elif operand == 5:
        return registers["B"]
    elif operand == 6:
        return registers["C"]
    

In [ ]:
def run_program(program, registers):
    instruction_pointer = 0
    output = []
    while True:
        if instruction_pointer > len(program) - 2:
            break

        opcode = program[instruction_pointer]
        operand = program[instruction_pointer + 1]

        if opcode == 0:
            numerator = registers["A"]
            denominator = 2 ** get_combo_operand(operand, registers)
            int_res = int(numerator / denominator)
            registers["A"] = int_res
        elif opcode == 1:
            registers["B"] = operand ^ registers["B"]
        elif opcode == 2:
            registers["B"] = get_combo_operand(operand, registers) % 8
        elif opcode == 3:
            if registers["A"] != 0:
                instruction_pointer = operand 
                continue 
        elif opcode == 4:
            registers["B"] = registers["B"] ^ registers["C"]
        elif opcode == 5:
            o = get_combo_operand(operand, registers) % 8
            output.append(o)
        elif opcode == 6:
            numerator = registers["A"]
            denominator = 2 ** get_combo_operand(operand, registers)
            int_res = int(numerator / denominator)
            registers["B"] = int_res
        elif opcode == 7:
            numerator = registers["A"]
            denominator = 2 ** get_combo_operand(operand, registers)
            int_res = int(numerator / denominator)
            registers["C"] = int_res

        instruction_pointer += 2
    return output

",".join([str(i) for i in run_program(program, deepcopy(registers))])

In [6]:
# part 2

In [7]:
# to get a 16 digit output, the number needs to be at least 2**45, 17 digit would be 2**48,
# that leaves 246290604621824 which is a bit much for brute force
# So I started looking for patterns in the lower numbers
# The last number in the output changes its value depending on the length of the output:

In [8]:
# len(output)==1  last number changes every 2**0
# len(output)==2  last number changes every 2**3
# len(output)==3  last number changes every 2**6
# len(output)==4  last number changes every 2**9
# ...
# len(output)==16  last number changes every 2**45

In [9]:
#last = 0
#for i in range(1, 16):
#    last+=3
#    print(i ,last)
# last

In [10]:
# len(output)==1  second to last number changes every -
# len(output)==2  second to last number changes every 2**0
# len(output)==3  second to last number changes every 2**3
# len(output)==4  second to last number changes every 2**6
# ...
# len(output)==16  second to last number changes every 2**42

In [11]:
# So the idea is to find the correct number by going through the back, starting where the output is already 16 digits 
# And then adding 2**((45-(3*checking_number_from_back))) to the register

# This got me very close, but not all the way as the second number changes multiple times
# I was too lazy to implement some backtracking and just did brute force from where almost all numbers matched
# Which was the last i output
# This worked

In [ ]:
i = 0
checking_number_from_back = 0 # count numbers from back
while True:
    regs = deepcopy(registers)
    regs["A"] = i
    output = run_program(program, regs)
    num_from_back_index = checking_number_from_back + 1
    if num_from_back_index > 14 and (output == program):
        print("Found solution", i)
        break
    if checking_number_from_back >= len(program):
        print("Trying to check too far. Last i:", i, output, program)
        break
    if output[-1 * num_from_back_index] == program[-1 * num_from_back_index]:
        checking_number_from_back += 1
        # Print the last few steps
        if num_from_back_index > 13:
            print("i", i, "checking from back", checking_number_from_back, "numbers:", output[-1 * num_from_back_index], program[-1 * num_from_back_index])
            print(output)
            print(program)
            print()
        continue 
    else:
        step_size = 2**((45-(3*checking_number_from_back)))
        # print("Stepsize", step_size)
        i += step_size

print(i)

In [ ]:
i = 0 # replace with <output> from above
while True:
    if i == 0:
        print("i needs to be replaced")
        break
    regs = deepcopy(registers)
    regs["A"] = i
    output = run_program(program, regs)
    if output == program:
        break
    i += 1
print(i) 